In [ ]:
from waymo_open_dataset import dataset_pb2

import json
import os

import numpy as np
import tensorflow as tf
from PIL import Image
from tqdm import tqdm
from waymo_open_dataset import label_pb2
from waymo_open_dataset.protos import camera_segmentation_pb2 as cs_pb2
from waymo_open_dataset.utils import box_utils


import matplotlib.pyplot as plt


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
from pathlib import Path


WOPD_DATA_ROOT = Path("/media/nvme1/waymo_perception/training")


tfrecords_file_list = list(WOPD_DATA_ROOT.glob("*.tfrecord"))

In [ ]:
import io
from pyquaternion import Quaternion

from d123.geometry import StateSE3
from d123.geometry.bounding_box import BoundingBoxSE3

from waymo_open_dataset.utils import frame_utils


# Frame attributes:
#   context: <class 'waymo_open_dataset.dataset_pb2.Context'>
#   timestamp_micros: <class 'int'>
#   pose: <class 'waymo_open_dataset.dataset_pb2.Transform'>
#   images: List with 5 images
#   lasers: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 5
#   laser_labels: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   projected_lidar_labels: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   camera_labels: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   no_label_zones: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   map_features: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   map_pose_offset: <class 'waymo_open_dataset.protos.vector_pb2.Vector3d'>

file_idx = 0
pathname = tfrecords_file_list[file_idx]
dataset = tf.data.TFRecordDataset(pathname, compression_type="")
num_frames = sum(1 for _ in dataset)


def read_jpg_image(data: bytes) -> np.ndarray:
    """Read a JPEG image from bytes and return it as a numpy array."""
    image = Image.open(io.BytesIO(data))
    return np.array(image)


ego_state_se3s = []
front_images = []
dataset = tf.data.TFRecordDataset(pathname, compression_type="")

boxes = []

for frame_idx, data in enumerate(dataset):

    frame = dataset_pb2.Frame()
    frame.ParseFromString(data.numpy())
    # print(frame.camera_labels)
    print("Frame attributes:")
    for field in frame.DESCRIPTOR.fields:
        field_name = field.name
        if hasattr(frame, field_name):
            value = getattr(frame, field_name)
            if field_name != "images":  # Don't print the whole image data
                print(f"  {field_name}: {type(value)}")
                if hasattr(value, "__len__") and not isinstance(value, (str, bytes)):
                    print(f"    Length: {len(value)}")
            else:
                print(f"  {field_name}: List with {len(value)} images")


    # # 1. pose
    pose = np.array(frame.pose.transform).reshape(4, 4)
    yaw_pitch_roll = Quaternion(matrix=pose[:3, :3]).yaw_pitch_roll
    ego_state_se3s.append(
        np.array(
            [
                pose[0, 3],  # x
                pose[1, 3],  # y
                pose[2, 3],  # z
                yaw_pitch_roll[2],  # yaw
                yaw_pitch_roll[1],  # pitch
                yaw_pitch_roll[0],  # roll
            ],
            dtype=np.float64,
        )
    )

    # # plt.show()
    if frame_idx == 0:
        break

ego_state_se3s = np.array(ego_state_se3s, dtype=np.float64)

In [ ]:
from typing import Dict, List, Tuple

import numpy as np
import tensorflow as tf

from waymo_open_dataset import dataset_pb2
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils

RangeImages = Dict["dataset_pb2.LaserName.Name", List[dataset_pb2.MatrixFloat]]
CameraProjections = Dict["dataset_pb2.LaserName.Name", List[dataset_pb2.MatrixInt32]]
SegmentationLabels = Dict["dataset_pb2.LaserName.Name", List[dataset_pb2.MatrixInt32]]
ParsedFrame = Tuple[RangeImages, CameraProjections, SegmentationLabels, dataset_pb2.MatrixFloat]


def parse_range_image_and_camera_projection(frame: dataset_pb2.Frame) -> ParsedFrame:
    """Parse range images and camera projections given a frame.

    Args:
      frame: open dataset frame proto

    Returns:
      range_images: A dict of {laser_name,
        [range_image_first_return, range_image_second_return]}.
      camera_projections: A dict of {laser_name,
        [camera_projection_from_first_return,
        camera_projection_from_second_return]}.
      seg_labels: segmentation labels, a dict of {laser_name,
        [seg_label_first_return, seg_label_second_return]}
      range_image_top_pose: range image pixel pose for top lidar.
    """
    range_images = {}
    camera_projections = {}
    seg_labels = {}
    range_image_top_pose: dataset_pb2.MatrixFloat = dataset_pb2.MatrixFloat()
    for laser in frame.lasers:
        if len(laser.ri_return1.range_image_compressed) > 0:  # pylint: disable=g-explicit-length-test
            range_image_str_tensor = tf.io.decode_compressed(laser.ri_return1.range_image_compressed, "ZLIB")
            ri = dataset_pb2.MatrixFloat()
            ri.ParseFromString(range_image_str_tensor.numpy())
            range_images[laser.name] = [ri]

            if laser.name == dataset_pb2.LaserName.TOP:
                range_image_top_pose_str_tensor = tf.io.decode_compressed(
                    laser.ri_return1.range_image_pose_compressed, "ZLIB"
                )
                range_image_top_pose = dataset_pb2.MatrixFloat()
                range_image_top_pose.ParseFromString(range_image_top_pose_str_tensor.numpy())

            camera_projection_str_tensor = tf.io.decode_compressed(
                laser.ri_return1.camera_projection_compressed, "ZLIB"
            )
            cp = dataset_pb2.MatrixInt32()
            cp.ParseFromString(camera_projection_str_tensor.numpy())
            camera_projections[laser.name] = [cp]

            if len(laser.ri_return1.segmentation_label_compressed) > 0:  # pylint: disable=g-explicit-length-test
                seg_label_str_tensor = tf.io.decode_compressed(laser.ri_return1.segmentation_label_compressed, "ZLIB")
                seg_label = dataset_pb2.MatrixInt32()
                seg_label.ParseFromString(seg_label_str_tensor.numpy())
                seg_labels[laser.name] = [seg_label]
        if len(laser.ri_return2.range_image_compressed) > 0:  # pylint: disable=g-explicit-length-test
            range_image_str_tensor = tf.io.decode_compressed(laser.ri_return2.range_image_compressed, "ZLIB")
            ri = dataset_pb2.MatrixFloat()
            ri.ParseFromString(range_image_str_tensor.numpy())
            range_images[laser.name].append(ri)

            camera_projection_str_tensor = tf.io.decode_compressed(
                laser.ri_return2.camera_projection_compressed, "ZLIB"
            )
            cp = dataset_pb2.MatrixInt32()
            cp.ParseFromString(camera_projection_str_tensor.numpy())
            camera_projections[laser.name].append(cp)

            if len(laser.ri_return2.segmentation_label_compressed) > 0:  # pylint: disable=g-explicit-length-test
                seg_label_str_tensor = tf.io.decode_compressed(laser.ri_return2.segmentation_label_compressed, "ZLIB")
                seg_label = dataset_pb2.MatrixInt32()
                seg_label.ParseFromString(seg_label_str_tensor.numpy())
                seg_labels[laser.name].append(seg_label)
    return range_images, camera_projections, seg_labels, range_image_top_pose


In [ ]:
from waymo_open_dataset.utils import frame_utils

dataset = tf.data.TFRecordDataset(tfrecords_file_list[file_idx], compression_type="")
for data in dataset:
    frame = dataset_pb2.Frame()
    frame.ParseFromString(data.numpy())  # No need for bytearray conversion
    break

(range_images, camera_projections, _, range_image_top_pose) = parse_range_image_and_camera_projection(frame)
points, cp_points = frame_utils.convert_range_image_to_point_cloud(
    frame=frame,
    range_images=range_images,
    camera_projections=camera_projections,
    range_image_top_pose=range_image_top_pose,
    keep_polar_features=True,
)


In [ ]:
pc = points[0]


plt.scatter(pc[:, 3], pc[:, 4], s=0.1, c=pc[:, 5], cmap='viridis')

In [ ]:
cp_points[0].shape, points[0].shape